Competition on Kaggle: https://www.kaggle.com/c/how-good-is-your-medium-article/leaderboard

In [1]:
!pip install tqdm

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import os
import codecs
import json
import seaborn as sns
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LassoLarsCV

from sklearn.linear_model import Ridge
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from datetime import date

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [3]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

In [4]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    ident = 0
    features = ['_id', 'content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding="utf-8")
                     for feat in features]
 
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:
    
        for line in tqdm_notebook(inp_json_file):
            dict_data = {}
            json_data = read_json_line(line)
            for i in range(len(feature_files)):
                dict_data[ident] = json_data[features[i]]
                feature_files[i].write(str(dict_data) + '\n')
            ident += 1
    for i in feature_files:
        i.close()

In [18]:
PATH_TO_DATA = 'C:\mlcourse\Databases\kaggle_medium'

In [6]:
extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

In [7]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

## Add the following groups of features:

- Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- Time features: publication hour, whether it's morning, day, night, whether it's a weekend
- Bag of authors (i.e. One-Hot-Encoded author names)

#### Extacting data

In [19]:
features = ['_id', 'content', 'published', 'title', 'author']

train_files = [os.path.join(PATH_TO_DATA,
                                       '{}_{}.txt'.format('train', feat))   
                     for feat in features]

test_files = [os.path.join(PATH_TO_DATA,
                                       '{}_{}.txt'.format('test', feat))   
                     for feat in features]

In [20]:
%%time
df_all_data = pd.DataFrame({})
for k in range(0, len(features)):
    f = codecs.open(train_files[k], "r", "utf-8")
    arr = []
    count = 0
    for line in f:
        count += 1
        arr.append(list(eval(line[:-2]).values())[0])
    f.close()
    ser_buf = pd.Series(arr)
    df_all_data[features[k]] = ser_buf

Wall time: 20min 52s


In [21]:
%%time
df_all_test = pd.DataFrame({})
for k in range(0, len(features)):
    f = codecs.open(test_files[k], "r", "utf-8")
    arr = []
    count = 0
    for line in f:
        count += 1
        arr.append(list(eval(line[:-2]).values())[0])
    f.close()
    ser_buf = pd.Series(arr)
    df_all_test[features[k]] = ser_buf

Wall time: 34min 44s


In [22]:
df_all_data['content'] = df_all_data['content'].map(strip_tags)
df_all_test['content'] = df_all_test['content'].map(strip_tags)


Adding a popularity column

In [23]:
data = pd.read_csv('C:\mlcourse\Databases\kaggle_medium/train_log1p_recommends.csv', sep=',')

In [24]:
df_all_data = df_all_data.join(data)

## Strip content: \xa0, \u200b

In [25]:
dddata = df_all_data.copy(deep=True)
dddata_test = df_all_test.copy(deep=True)

In [26]:
df_all_data = dddata.copy(deep=True)
df_all_test = dddata_test.copy(deep=True)

## Division publications by hour, whether it's morning, day, night, whether it's a weekend


In [27]:
df_all_data.head()

,_id,content,published,title,author,id,log_recommends
0,https://medium.com/policy/medium-terms-of-serv...,"Medium Everyone’s stories and ideas Aug 13, 20...",{'$date': '2012-08-13T22:54:53.510Z'},Medium Terms of Service – Medium Policy – Medium,"{'name': None, 'url': 'https://medium.com/@Med...",8,9.01201
1,https://medium.com/policy/amendment-to-medium-...,"Medium Everyone’s stories and ideas Aug 2, 201...",{'$date': '2015-08-03T07:44:50.331Z'},Amendment to Medium Terms of Service Applicabl...,"{'name': None, 'url': 'https://medium.com/@Med...",14,3.49651
2,https://medium.com/@aelcenganda/%E9%96%A9%E6%9...,"Yun-Chen Chien（簡韻真） Nobody in @g0v.tw, PM in s...",{'$date': '2017-02-05T13:08:17.410Z'},走入山與海之間：閩東大刀會和兩岸走私 – Yun-Chen Chien（簡韻真） – Medium,"{'name': None, 'url': 'https://medium.com/@ael...",19,0.69315
3,https://medium.com/what-comes-to-mind/how-fast...,Vaibhav Khulbe Android App Developer | I write...,{'$date': '2017-05-06T08:16:30.776Z'},How fast can a camera get? – What comes to min...,"{'name': None, 'url': 'https://medium.com/@vai...",22,1.38629
4,https://medium.com/what-comes-to-mind/a-game-f...,Vaibhav Khulbe Android App Developer | I write...,{'$date': '2017-06-04T14:46:25.772Z'},A game for the lonely fox – What comes to mind...,"{'name': None, 'url': 'https://medium.com/@vai...",29,1.94591


#### Creating the columns date and time and preprocessing author

In [28]:
def pub_to_time(publicate):
    return publicate[11:-5]

def pub_to_date(publicate):
    return publicate[:10] 

def time_to_sec(time_s):
    return ((int(time_s[:2])* 60 * 60) + (int(time_s[3:5])) * 60 + (int(time_s[6:])))

# may be needed other division
def sec_to_tofd(time_sec):
    x = 3600
    if(time_sec > 18 * x):
        return 'E'
    elif(time_sec > 12 * x):
        return 'D'
    elif(time_sec > 6 * x):
        return 'M'
    else:
        return 'N'
    
def day_of_week(d):
    days = date(int(d[:4]), int(d[5:7]), int(d[8:]))
    return(days.weekday())

def is_weekend(d):
    if (d > 4):
        return True
    return False

def twitter_data(author_data):
    return author_data['twitter']

def url_data(author_data):
    return author_data['url']

def publish_to_str(publ):
    return list(publ.values())[0]

In [29]:
df_all_data['published'] = df_all_data['published'].map(publish_to_str)
df_all_data['date'] = df_all_data['published'].map(pub_to_date)
df_all_data['time'] = df_all_data['published'].map(pub_to_time)
df_all_data['time_sec'] = df_all_data['time'].map(time_to_sec)
df_all_data['time_of_day'] = df_all_data['time_sec'].map(sec_to_tofd)
df_all_data['week_day'] = df_all_data['date'].map(day_of_week)
df_all_data['weekend'] = df_all_data['week_day'].map(is_weekend)
df_all_data['twitter'] = df_all_data['author'].map(twitter_data)
df_all_data['url'] = df_all_data['author'].map(url_data)
df_all_data['cont_len']= df_all_data['content'].map(len)
df_all_data['title_len']= df_all_data['title'].map(len)

df_all_test['published'] = df_all_test['published'].map(publish_to_str)
df_all_test['date'] = df_all_test['published'].map(pub_to_date)
df_all_test['time'] = df_all_test['published'].map(pub_to_time)
df_all_test['time_sec'] = df_all_test['time'].map(time_to_sec)
df_all_test['time_of_day'] = df_all_test['time_sec'].map(sec_to_tofd)
df_all_test['week_day'] = df_all_test['date'].map(day_of_week)
df_all_test['weekend'] = df_all_test['week_day'].map(is_weekend)
df_all_test['twitter'] = df_all_test['author'].map(twitter_data)
df_all_test['url'] = df_all_test['author'].map(url_data)
df_all_test['cont_len']= df_all_test['content'].map(len)
df_all_test['title_len']= df_all_test['title'].map(len)

In [30]:
# Not here
feats = ['time_of_day', 'week_day','cont_len', 'title_len', 'log_recommends']
df = df_all_data[feats]
df_test = df_all_test[feats[:-1]]
#X_train, y_train = train_test_split(df, random_state=17)

In [31]:
df_all_test.head()

,_id,content,published,title,author,date,time,time_sec,time_of_day,week_day,weekend,twitter,url,cont_len,title_len
0,https://medium.com/on-mornings/nocturnalmornin...,Member preview HITRECORD hitrecord.org is a ne...,2018-02-21T17:01:00.928Z,"For Night Owls, the Day Starts with a Nocturna...","{'name': None, 'url': 'https://medium.com/@HIT...",2018-02-21,17:01:00,61260,D,2,False,@hitRECord,https://medium.com/@HITRECORD.org,12580,55
1,https://medium.com/wordsthatmatter/never-break...,Member preview Maria Bustillos is a journalist...,2017-12-07T15:56:00.967Z,Blockchain is Memory – Words That Matter – Medium,"{'name': None, 'url': 'https://medium.com/@mar...",2017-12-07,15:56:00,57360,D,3,False,@mariabustillos,https://medium.com/@mariabustillos,6759,49
2,https://medium.com/on-mornings/onmorningscredi...,HITRECORD hitrecord.org is a new kind of onlin...,2018-02-08T17:33:27.816Z,ON MORNINGS Credits – On Mornings – Medium,"{'name': None, 'url': 'https://medium.com/@HIT...",2018-02-08,17:33:27,63207,D,3,False,@hitRECord,https://medium.com/@HITRECORD.org,7337,42
3,https://medium.com/@LanceUlanoff/apple-homepod...,"Member preview Lance Ulanoff Tech expert, jour...",2018-02-09T18:04:12.262Z,Apple HomePod Review: Almost love – Lance Ulan...,"{'name': None, 'url': 'https://medium.com/@Lan...",2018-02-09,18:04:12,65052,E,4,False,@LanceUlanoff,https://medium.com/@LanceUlanoff,13027,58
4,https://blog.medium.com/tips-and-tricks-for-me...,"Medium Everyone’s stories and ideas Oct 2, 201...",2017-10-02T23:08:00.000Z,Tips and tricks for Medium writers – 3 min read,"{'name': None, 'url': 'https://blog.medium.com...",2017-10-02,23:08:00,83280,E,0,False,@Medium,https://blog.medium.com/@Medium,5723,47


#### Creating 'Bag of authors'

In [32]:
auth_set = df_all_data['url'].append(df_all_test['url'])

In [33]:
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False)
labels_authors = label_encoder.fit(auth_set)
df_all_data['url'] = label_encoder.transform(df_all_data['url'])
df_all_test['url'] = label_encoder.transform(df_all_test['url'])


In [34]:
author_encoded_cols_test = pd.DataFrame(onehot_encoder.fit_transform((df_all_test['url'].values).reshape(-1, 1)))

In [36]:
author_encoded_categorical_columns = pd.DataFrame(onehot_encoder.fit_transform((df_all_data['url'].values).reshape(-1, 1)))

MemoryError: 

In [37]:
author_encoded_cols_test.shape, author_encoded_categorical_columns.shape

NameError: name 'author_encoded_categorical_columns' is not defined

#### Creating 'Bag of time and week days'

In [38]:
for i in feats[:2]:
    df[i] = label_encoder.fit_transform(df[i])
data_time_train = pd.DataFrame(onehot_encoder.fit_transform((df[feats[:2]])))
data_train = data_time_train.join(df[feats[2:-1]])

for i in feats[:2]:
    df_test[i] = label_encoder.fit_transform(df_test[i])
data_time_test = pd.DataFrame(onehot_encoder.fit_transform((df_test[feats[:2]])))
data_test = data_time_train.join(df_test[feats[2:-1]])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [39]:
data_train.head()

,0,1,2,3,4,5,6,7,8,9,10,cont_len,title_len
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7007,48
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,10817,72
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4588,49
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4436,56
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5474,55


In [40]:
data_test.head()

,0,1,2,3,4,5,6,7,8,9,10,cont_len,title_len
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12580.0,55.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6759.0,49.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7337.0,42.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13027.0,58.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5723.0,47.0


## Creating Tf-Idf 

- with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
- with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)


### train data

In [41]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_title = count_vect.fit_transform(df_all_data.title)

tf_transformer_title = TfidfTransformer(use_idf=False).fit(df_counts_title)
df_tf_title = tf_transformer_title.transform(df_counts_title)

tfidf_transformer = TfidfTransformer()
df_tfidf_title = tfidf_transformer.fit_transform(df_counts_title)

Wall time: 8.53 s


In [49]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_content = count_vect.fit_transform(df_all_data.content)

#tf_transformer_content = TfidfTransformer(use_idf=False).fit(df_counts_content)
#df_tf_content = tf_transformer_content.transform(df_counts_content)

tfidf_transformer = TfidfTransformer()
df_tfidf_content = tfidf_transformer.fit_transform(df_counts_content)

MemoryError: 

### test data

In [43]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_title_t = count_vect.fit_transform(df_all_test.title)

tf_transformer_title = TfidfTransformer(use_idf=False).fit(df_counts_title_t)
df_tf_title_te = tf_transformer_title.transform(df_counts_title_t)

tfidf_transformer = TfidfTransformer()
df_tfidf_title_te = tfidf_transformer.fit_transform(df_counts_title_t)

Wall time: 21.8 s


In [48]:
%%time

count_vect = CountVectorizer(ngram_range=(1, 2), max_features= 100000)
df_counts_content_t = count_vect.fit_transform(df_all_test.content)

tfidf_transformer = TfidfTransformer()
df_tfidf_content_te = tfidf_transformer.fit_transform(df_counts_content_t)

Wall time: 10min 28s


In [45]:
df_tfidf_content.shape, df_tfidf_title.shape, author_encoded_categorical_columns.shape

NameError: name 'df_tfidf_content' is not defined

## Sparsing train and test data

In [46]:
%%time
X_train_sparse = hstack([author_encoded_categorical_columns,
                         df_tfidf_content, df_tfidf_title, data_train]).tocsr()

NameError: name 'author_encoded_categorical_columns' is not defined

In [47]:
%%time
X_test_sparse = hstack([author_encoded_cols_test,
                         df_tfidf_content_te, df_tfidf_title_te, data_test]).tocsr()

NameError: name 'df_tfidf_content_te' is not defined

In [ ]:
y_train = df[feats[-1]]

In [ ]:
X_train_sparse.shape, len(df[feats[-1]]), X_test_sparse.shape

 Split data on train and validation sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train_sparse, df[feats[-1]], random_state=17)

In [104]:
train_part_size = int(0.7 * y_train.shape[0])
X_train_part_sparse = X_train_sparse[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid_sparse =  X_train_sparse[train_part_size:, :]
y_valid = y_train[train_part_size:]

Using different models and check MAE on validation set

In [ ]:
def mean_err(arr1, arr2, p = 1):
    if (len(arr1) == len(arr2)):
        sum = 0
        for i in range(len(arr1)):
            sum += (abs(arr1[i] - arr2[i])) ** p
        return ((sum ** (1 / p)) / len(arr1))
    return(0)

Ridge model

In [ ]:
%%time
for i in range(3, 6):
    ridge = Ridge(alpha = i / 4)
    ridge.fit(X_train, y_train)
    print("alpha: ", i / 4, " MAE: ", ridge.score(X_test, y_test), mean_err(ridge.predict(X_test), y_test.values))

In [ ]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join(PATH_TO_DATA, 
                                                      'sample_submission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='id')
    
    submission['log_recommends'] = prediction
    submission.to_csv(filename)

In [ ]:
write_submission_file(ridge_test_pred, os.path.join(PATH_TO_DATA,
                                                    'assignment6_medium_submission.csv'))

dirty Kaggle hacks. Form a submission file with all zeros. Make a submission. What do you get if you think about it? How is it going to help you with modifying your predictions?

In [ ]:
write_submission_file(np.zeros_like(ridge_test_pred), 
                      os.path.join(PATH_TO_DATA,
                                   'medium_all_zeros_submission.csv'))

Modify predictions in an appropriate way (based on your all-zero submission) and make a new submission.

In [ ]:
ridge_test_pred_modif = ridge_test_pred # You code here


In [ ]:
write_submission_file(ridge_test_pred_modif, 
                      os.path.join(PATH_TO_DATA,
                                   'assignment6_medium_submission_with_hack.csv'))
